In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler


c:\venv_python_3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import xgboost as xgb
import lightgbm as lgbm
import catboost as cbt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Iris Dataset

In [3]:
# import some data to play with
iris = datasets.load_iris()
# Take the first two features. We could avoid this by using a two-dim dataset
# X = iris.data[:, :2]
# y = iris.target


X = iris.data
y = iris.target

In [4]:
print("X shape")
print(X.shape)
print("Y shape")
print(y.shape)


X shape
(150, 4)
Y shape
(150,)


In [5]:
# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4096)
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4096, train_size=0.8)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=4096, train_size=0.8)

In [6]:
scaler = MinMaxScaler()

scaler = scaler.fit(x_train)


In [7]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

In [8]:
# data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
# target = raw_df.values[1::2, 2]
# x_train, x_test, y_train, y_test = train_test_split(data, target, random_state=4096)

# SVM - Linear

In [9]:

svm_classifier = SVC(kernel='linear')
svm_classifier = svm_classifier.fit(x_train, y_train)

In [10]:
y_pred = svm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))


Confusion Matrix
 [[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]
Accuracy  0.9666666666666667


# SVM - Kernel

In [11]:
param_grid = [
    {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
    {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
                        'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

In [12]:

svm_classifier = SVC(kernel='rbf')
# svm_classifier = svm_classifier.fit(x_train, y_train)

grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_svm_classifier = grid_search.fit(x_train, y_train)



Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END .............................C=100.0, kernel=linear; total time=   0.0s
[CV] END .............................C=100.0, 

In [13]:
grid_search.best_params_


{'C': 10.0, 'kernel': 'linear'}

In [14]:
# y_pred = svm_classifier.predict(x_test)
y_pred = best_svm_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]
Accuracy  0.9666666666666667


# TabNet

In [41]:
tabnet_classifier = TabNetClassifier()
# clf = TabNetMultiTaskClassifier()
tabnet_classifier.fit(
  X_train=x_train, y_train=y_train,
  eval_set = [(x_valid, y_valid)],
  patience=1000,
  max_epochs=60,
  batch_size=16,
)

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.97386 | val_0_accuracy: 0.375   |  0:00:00s
epoch 1  | loss: 0.42695 | val_0_accuracy: 0.375   |  0:00:00s
epoch 2  | loss: 0.33122 | val_0_accuracy: 0.375   |  0:00:00s
epoch 3  | loss: 0.21641 | val_0_accuracy: 0.375   |  0:00:01s
epoch 4  | loss: 0.44749 | val_0_accuracy: 0.375   |  0:00:01s
epoch 5  | loss: 0.26021 | val_0_accuracy: 0.45833 |  0:00:01s
epoch 6  | loss: 0.17368 | val_0_accuracy: 0.75    |  0:00:01s
epoch 7  | loss: 0.17839 | val_0_accuracy: 0.75    |  0:00:02s
epoch 8  | loss: 0.11249 | val_0_accuracy: 0.79167 |  0:00:02s
epoch 9  | loss: 0.16401 | val_0_accuracy: 0.625   |  0:00:02s
epoch 10 | loss: 0.2172  | val_0_accuracy: 0.79167 |  0:00:03s
epoch 11 | loss: 0.08778 | val_0_accuracy: 0.70833 |  0:00:03s
epoch 12 | loss: 0.13652 | val_0_accuracy: 0.83333 |  0:00:03s
epoch 13 | loss: 0.13801 | val_0_accuracy: 0.75    |  0:00:03s
epoch 14 | loss: 0.16824 | val_0_accuracy: 0.79167 |  0:00:04s
epoch 15 | loss: 0.19965 | val_0_accuracy: 0.91667 |  0

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [42]:
y_pred = tabnet_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]
Accuracy  0.9666666666666667


# XGBoost

# XGBoost

In [21]:
num_round = 3
xgb_classifier = xgb.XGBClassifier()

xgb_parameters ={'max_depth' : [3,4,5,6] , 'n_estimators': [12,24,32], 
 'learning_rate' : [0.01,0.02,0.03],
 'gamma': [0.5, 1, 2], 'random_state':[99]}

In [22]:
# xgb_classifier=xgb_classifier.fit(x_train, y_train)

xgb_grid_search = GridSearchCV(xgb_classifier, xgb_parameters, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_xgb_classifier = xgb_grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; to

In [23]:

xgb_grid_search.best_params_

{'gamma': 0.5,
 'learning_rate': 0.02,
 'max_depth': 3,
 'n_estimators': 32,
 'random_state': 99}

In [24]:
# y_pred = xgb_classifier.predict(x_test)
y_pred = best_xgb_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]
Accuracy  0.9666666666666667


# LightGBM

In [25]:
num_round = 3
lgbm_classifier = lgbm.LGBMClassifier()

In [26]:
lgbm_classifier=lgbm_classifier.fit(x_train, y_train)

In [27]:

y_pred = lgbm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]
Accuracy  0.9666666666666667


# CatBoost

In [28]:
num_round = 3
cbt_classifier = cbt.CatBoostClassifier()

In [29]:
cbt_parameters = {'depth'         : [3,4,5,6],
                'learning_rate' : [0.01,0.02,0.03],
                'n_estimators': [12,24,32], 
}

In [30]:
# cbt_classifier=cbt_classifier.fit(x_train, y_train)

cbt_grid_search = GridSearchCV(cbt_classifier, cbt_parameters, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_cbt_classifier = cbt_grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 1.0866661	total: 136ms	remaining: 1.5s
1:	learn: 1.0775959	total: 136ms	remaining: 682ms
2:	learn: 1.0656883	total: 137ms	remaining: 410ms
3:	learn: 1.0544958	total: 137ms	remaining: 274ms
4:	learn: 1.0433123	total: 137ms	remaining: 192ms
5:	learn: 1.0330824	total: 138ms	remaining: 138ms
6:	learn: 1.0245681	total: 138ms	remaining: 98.5ms
7:	learn: 1.0136948	total: 138ms	remaining: 69ms
8:	learn: 1.0029695	total: 138ms	remaining: 46.1ms
9:	learn: 0.9954930	total: 138ms	remaining: 27.7ms
10:	learn: 0.9871990	total: 139ms	remaining: 12.6ms
11:	learn: 0.9762604	total: 139ms	remaining: 0us
[CV] END .......depth=3, learning_rate=0.01, n_estimators=12; total time=   0.1s
0:	learn: 1.0892380	total: 275us	remaining: 3.03ms
1:	learn: 1.0778659	total: 470us	remaining: 2.35ms
2:	learn: 1.0660765	total: 648us	remaining: 1.95ms
3:	learn: 1.0545795	total: 818us	remaining: 1.64ms
4:	learn: 1.0466511	total: 988us	remaining: 1.38ms


In [31]:
cbt_grid_search.best_params_

{'depth': 3, 'learning_rate': 0.01, 'n_estimators': 12}

In [32]:
# y_pred = cbt_classifier.predict(x_test)
y_pred = best_cbt_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]
Accuracy  0.9666666666666667


# Random Forest

In [33]:
rf_param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7],
    'criterion' :['gini', 'entropy']
}

In [34]:
num_round = 3
rf_classifier = RandomForestClassifier()

rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_rf_classifier = rf_grid_search.fit(x_train, y_train)



Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s
[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s
[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.0s


...

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=7, max_features=auto, n_estimators=500; total time=   0.3s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=7, max_features=auto, n_estimators=500; total time=   0.3s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=200; total time=   0.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   0.3s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   0.2s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   0.2s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, n_estimators=500; total time=   0.3s
[CV] END criterion=entropy, ma

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [38]:
# rf_classifier = rf_classifier.fit(x_train, y_train)

In [39]:
best_rf_classifier.best_params_

{'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'auto',
 'n_estimators': 200}

In [40]:

y_pred = best_rf_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[ 9  0  0]
 [ 0 13  1]
 [ 0  0  7]]
Accuracy  0.9666666666666667
